<a href="https://colab.research.google.com/github/yhpark1962/baseball/blob/main/news_crawling_Best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 설치 및 가져오기
!pip install requests
!pip install pandas
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from datetime import datetime

# 검색어 입력
query = input("keyword to search: ")

# Google 뉴스 검색 URL (start 파라미터로 페이지 변경)
base_url = f"https://www.google.com/search?q={query}&tbm=nws"

# HTTP 요청 헤더 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"
}

# 뉴스 데이터 추출
titles = []
contents = []
links = []
dates = []

# 원하는 뉴스 갯수 설정
max_news = 40
page = 0

while len(titles) < max_news:
    # start 파라미터로 여러 페이지 크롤링
    url = f"{base_url}&start={page * 10}"

    # HTTP GET 요청 보내기
    response = requests.get(url, headers=headers)
    soup = bs(response.text, "html.parser")

    # 뉴스 아이템 선택
    news_items = soup.select(".SoaBEf")

    for item in news_items:
        title = item.select_one(".nDgy9d").text if item.select_one(".nDgy9d") else "제목 없음"
        content = item.select_one(".Y3v8qd").text if item.select_one(".Y3v8qd") else "본문 없음"
        link = item.select_one("a").get("href") if item.select_one("a") else "링크 없음"

       # 날짜 추출 부분 수정
date_text = item.select_one(".WG9SHc")
if date_text:
    date_text = date_text.text.strip()
else:
    date_text = None

# 날짜가 없으면 다른 방법으로 추출 (구글 뉴스의 형식에 따라 다를 수 있음)
if date_text is None:
    date_text = item.select_one(".rc .slp .MUxGbd")  # 날짜가 있을 수 있는 다른 태그 찾기
    if date_text:
        date_text = date_text.text.strip()

# 출력해봐서 어떤 형식으로 날짜가 추출되는지 확인
print("Date text:", date_text)

# 날짜 형식이 '오늘'이면 오늘 뉴스로 분류
if date_text and "오늘" in date_text:
    news_date = datetime.today().date()
else:
    if date_text:  # date_text가 None이 아닐 경우에만 처리
        try:
            # 날짜가 '오늘'이 아니면 다양한 형식 (2025.03.19, 2025-03-19, 2025/03/19)을 처리
            news_date = datetime.strptime(date_text, "%Y.%m.%d").date()  # 2025.03.19 형식
        except ValueError:
            try:
                news_date = datetime.strptime(date_text, "%Y-%m-%d").date()  # 2025-03-19 형식
            except ValueError:
                try:
                    news_date = datetime.strptime(date_text, "%Y/%m/%d").date()  # 2025/03/19 형식
                except ValueError:
                    news_date = None
    else:
        news_date = None


        # 제목, 본문, 링크, 날짜 추가
        titles.append(title)
        contents.append(content)
        links.append(link)
        dates.append(news_date)

        # 최대 뉴스 갯수에 도달하면 종료
        if len(titles) >= max_news:
            break

    page += 1  # 다음 페이지로 이동

# 데이터프레임 생성
news_df = pd.DataFrame({
    "Titles": titles,
    "Contents": contents,
    "Links": links,
    "Date": dates
})

# 최신 뉴스가 먼저 오도록 날짜 기준으로 정렬
news_df = news_df.sort_values(by="Date", ascending=False)

# Colab 환경에서 파일 저장 경로 설정
output_path = "/content/news_crawling.csv"

# CSV로 저장
news_df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"news data saved: {output_path}")

# 데이터 미리보기
news_df.head()


keyword to search: 양자 컴퓨터
